## Workshop - Regularization

In this workshop, we are going to:

1. Tune an elastic-net regression 
2. Compare the following models:
    1. The null model
    2. The tuned elastic-net model
    3. The trimmed non-regularized model with standardized features
    4. The trimmed non-regularized model with non-standardized features
    
# Preliminaries

- Load any necessary packages and/or functions
- Load in and prepare the class data
- Create x and y with a label of `pct_d_rgdp`
- Create `x_train`, `x_test`, `y_train`, `y_test` with
    * training size of two-thirds
    * random state of 490
- Standardize the features
- Add constants

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import linear_model as lm

In [2]:
df = pd.read_pickle("C:/Users/dp846/OneDrive/Desktop/ECON490ML/class data/class_data.pkl")
df

pct_d_rgdp urate_bin  pos_net_jobs  emp_estabs  \
fips  year GeoName                                                       
1001  2002 Autauga, AL    3.202147     lower             1   12.531208   
      2003 Autauga, AL    1.434404     lower             1   12.598415   
      2004 Autauga, AL   15.061365     lower             1   12.780078   
      2005 Autauga, AL    0.333105    higher             1   12.856784   
      2006 Autauga, AL    7.440034    higher             1   12.832506   
...                            ...       ...           ...         ...   
56045 2014 Weston, WY     2.055429     lower             1    8.410000   
      2015 Weston, WY    12.558802     lower             0    8.415385   
      2016 Weston, WY   -10.381257   similar             0    7.644231   
      2017 Weston, WY    -0.153371     lower             0    7.808081   
      2018 Weston, WY     7.624480     lower             0    8.074468   

                        estabs_entry_rate  estabs_exit_rate      pop  \
fips  year GeoName                                                     
1001  2002 Autauga, AL             11.268             9.256  45909.0   
      2003 Autauga, AL             10.603             9.940  46800.0   
      2004 Autauga, AL             11.140             8.519  48366.0   
      2005 Autauga, AL             11.735             8.673  49676.0   
      2006 Autauga, AL             10.645             8.766  51328.0   
...                                   ...               ...      ...   
56045 2014 Weston, WY               9.694             5.612   7138.0   
      2015 Weston, WY               6.076             8.608   7208.0   
      2016 Weston, WY              13.896             7.444   7220.0   
      2017 Weston, WY               5.941             9.901   6968.0   
      2018 Weston, WY               5.208             9.375   6924.0   

                        pop_pct_black  pop_pct_hisp       lfpr    density  \
fips  year GeoName                                                          
1001  2002 Autauga, AL      17.386569      1.611884  74.841638  77.231178   
      2003 Autauga, AL      17.493590      1.692308  75.093851  78.730077   
      2004 Autauga, AL      17.584667      1.796717  74.459624  81.364507   
      2005 Autauga, AL      17.612127      1.986875  74.920228  83.568276   
      2006 Autauga, AL      17.898613      2.032029  73.641001  86.347380   
...                               ...           ...        ...        ...   
56045 2014 Weston, WY        1.204819      3.950686  87.627044   2.976537   
      2015 Weston, WY        1.054384      3.953940  86.978480   3.005727   
      2016 Weston, WY        1.038781      4.099723  87.816245   3.010731   
      2017 Weston, WY        1.248565      4.118829  87.065369   2.905647   
      2018 Weston, WY        1.299827      3.942808  89.519962   2.887299   

                        year  
fips  year GeoName            
1001  2002 Autauga, AL  2002  
      2003 Autauga, AL  2003  
      2004 Autauga, AL  2004  
      2005 Autauga, AL  2005  
      2006 Autauga, AL  2006  
...                      ...  
56045 2014 Weston, WY   2014  
      2015 Weston, WY   2015  
      2016 Weston, WY   2016  
      2017 Weston, WY   2017  
      2018 Weston, WY   2018  

[50834 rows x 12 columns]

In [3]:
df_prepped = df.drop(columns = ['urate_bin', 'year']).join([
    pd.get_dummies(df['urate_bin'], drop_first = True),
    pd.get_dummies(df.year, drop_first = True)    
])

In [4]:
y = df_prepped['pct_d_rgdp']
x = df_prepped.drop(columns = 'pct_d_rgdp')

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 2/3, random_state = 490)

x_train_std = x_train.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)
x_test_std  = x_test.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)

x_train_std = sm.add_constant(x_train_std)
x_test_std  = sm.add_constant(x_test_std)
x_train     = sm.add_constant(x_train)
x_test      = sm.add_constant(x_test)

Take a look at `lm.ElasticNet?` and 
```
fit = sm.OLS(y_train, x_train)
fit.fit_regularized?
```
Determine which coefficients are the same, but named differently.
Specifically, $\alpha$ and the weight on the different constraints (i.e. $||\beta||_2$ and $||\beta||_1$).

In [5]:
# lm.ElasticNet?
fit = sm.OLS(y_train, x_train)
fit.fit_regularized?

Signature:
fit.fit_regularized(
    method='elastic_net',
    alpha=0.0,
    L1_wt=1.0,
    start_params=None,
    profile_scale=False,
    refit=False,
    **kwargs,
)
Docstring:
Return a regularized fit to a linear regression model.

Parameters
----------
method : str
    Either 'elastic_net' or 'sqrt_lasso'.
alpha : scalar or array_like
    The penalty weight.  If a scalar, the same penalty weight
    applies to all variables in the model.  If a vector, it
    must have the same length as `params`, and contains a
    penalty weight for each coefficient.
L1_wt : scalar
    The fraction of the penalty given to the L1 penalty term.
    Must be between 0 and 1 (inclusive).  If 0, the fit is a
    ridge fit, if 1 it is a lasso fit.
start_params : array_like
    Starting values for ``params``.
profile_scale : bool
    If True the penalized fit is computed using the profile
    (concentrated) log-likelihood for the Gaussian model.
    Otherwise the fit uses the residual sum of squares.
ref

Perform a 5-fold cross-validation grid search with a random state of 490. 
Identify the optimally tuned hyperparameters.
Use this grid:
```
param_grid = {'alpha': 10.**np.arange(-5, -1, 1), 
              'l1_ratio': np.arange(0, 1, 0.1)}
```
You will get a warning message about convergence.
We will discuss it after the workshop.
Think about why it occuring.

****
# Question

How many models did we just fit?

***
Using the tuned hyperparameters, fit your elastic net model with `statsmodels`

Using the selected features refit

- the non-regularized model with standardized features
- the non-regularized model with non-standardized features

Compare the percent improvement from the null model RMSE to the elastic-net and OLS model.